In [ ]:
# Celda 1: Importaciones
!pip install gradio -q
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
import gradio as gr

In [ ]:
# Celda 2: Definición de la clase BioprocessModel
class BioprocessModel:
    def __init__(self):
        self.params = {}
        self.r2 = {}
        self.rmse = {}
        self.datax = []
        self.datas = []
        self.datap = []
        self.dataxp = []
        self.datasp = []
        self.datapp = []
        self.dataxi = []
        self.datasi = []
        self.datapi = []
        self.dataxf = []
        self.datasf = []
        self.datapf = []
        self.dataxit = []
        self.datasit = []
        self.datapit = []
        self.dataxft = []
        self.datasft = []
        self.datapft = []

    @staticmethod
    def logistic(time, xo, xm, um):
        return (xo * np.exp(um * time)) / (1 - (xo / xm) * (1 - np.exp(um * time)))

    @staticmethod
    def substrate(time, so, p, q, xo, xm, um):
        return so - (p * xo * ((np.exp(um * time)) / (1 - (xo / xm) * (1 - np.exp(um * time))) - 1)) - \
               (q * (xm / um) * np.log(1 - (xo / xm) * (1 - np.exp(um * time))))

    @staticmethod
    def product(time, po, alpha, beta, xo, xm, um):
        return po + (alpha * xo * ((np.exp(um * time) / (1 - (xo / xm) * (1 - np.exp(um * time)))) - 1)) + \
               (beta * (xm / um) * np.log(1 - (xo / xm) * (1 - np.exp(um * time))))

    @staticmethod
    def gompertz_biomass(time, xo, lbd, um, bg):
        return xo * np.exp(((lbd * um) / (bg - 1)) * (1 - np.exp(-((bg - 1) * time) / lbd)))

    @staticmethod
    def gompertz_product(time, po, um, cg):
        return po * (np.exp((um / cg) * (1 - np.exp(-cg * time))) - 1)

    @staticmethod
    def gompertz_substrate(time, so, po, yps, bg, cg):
        return so - (po / yps) * (np.exp(np.exp(bg) - np.exp(bg - cg * time)) - 1)

    @staticmethod
    def moser_biomass(time, xo, umax, ks, n, sustrato):
        return xo * (np.exp(1) ** ((umax * (sustrato ** n) * time) / (ks + (sustrato ** n))))

    @staticmethod
    def moser_substrate(time, so, p, xo, umax, ks, n, sustrato):
        return (so-(p*(xo*(np.exp(1)**((umax*(sustrato**(n))*time)/(ks+(sustrato**(n))))))))

    @staticmethod
    def moser_product(time, po, p, xo, umax, ks, n, sustrato):
        return (po+(p*(xo*(np.exp(1)**((umax*(sustrato**(n))*time)/(ks+(sustrato**(n))))))))

In [ ]:
# Celda 3: Métodos de procesamiento y ajuste de datos
class BioprocessModel(BioprocessModel):  # Continuación de la clase
    def process_data(self, df):
        Biomasa_A = df['A']['Biomasa']
        Biomasa_B = df['B']['Biomasa']
        Biomasa_C = df['C']['Biomasa']

        data0 = [*Biomasa_A, *Biomasa_B, *Biomasa_C]
        self.datax.append(data0)

        self.dataxp.append(np.mean([Biomasa_A, Biomasa_B, Biomasa_C], axis=0))
        self.dataxi.append(np.mean([Biomasa_A.iloc[0], Biomasa_B.iloc[0], Biomasa_C.iloc[0]]))
        self.dataxf.append(np.mean([Biomasa_A.iloc[-1], Biomasa_B.iloc[-1], Biomasa_C.iloc[-1]]))
        self.dataxit.append([Biomasa_A.iloc[0], Biomasa_B.iloc[0], Biomasa_C.iloc[0]])
        self.dataxft.append([Biomasa_A.iloc[-1], Biomasa_B.iloc[-1], Biomasa_C.iloc[-1]])

        Sustrato_A = df['A']['Sustrato']
        Sustrato_B = df['B']['Sustrato']
        Sustrato_C = df['C']['Sustrato']

        data0 = [*Sustrato_A, *Sustrato_B, *Sustrato_C]
        self.datas.append(data0)

        self.datasp.append(np.mean([Sustrato_A, Sustrato_B, Sustrato_C], axis=0))
        self.datasi.append(np.mean([Sustrato_A.iloc[0], Sustrato_B.iloc[0], Sustrato_C.iloc[0]]))
        self.datasf.append(np.mean([Sustrato_A.iloc[-1], Sustrato_B.iloc[-1], Sustrato_C.iloc[-1]]))
        self.datasit.append([Sustrato_A.iloc[0], Sustrato_B.iloc[0], Sustrato_C.iloc[0]])
        self.datasft.append([Sustrato_A.iloc[-1], Sustrato_B.iloc[-1], Sustrato_C.iloc[-1]])

        Producto_A = df['A']['Producto']
        Producto_B = df['B']['Producto']
        Producto_C = df['C']['Producto']

        data0 = [*Producto_A, *Producto_B, *Producto_C]
        self.datap.append(data0)

        self.datapp.append(np.mean([Producto_A, Producto_B, Producto_C], axis=0))
        self.datapi.append(np.mean([Producto_A.iloc[0], Producto_B.iloc[0], Producto_C.iloc[0]]))
        self.datapf.append(np.mean([Producto_A.iloc[-1], Producto_B.iloc[-1], Producto_C.iloc[-1]]))
        self.datapit.append([Producto_A.iloc[0], Producto_B.iloc[0], Producto_C.iloc[0]])
        self.datapft.append([Producto_A.iloc[-1], Producto_B.iloc[-1], Producto_C.iloc[-1]])

    def fit_model(self, model_type='logistic'):
        if model_type == 'logistic':
            self.fit_biomass = self.fit_biomass_logistic
            self.fit_substrate = self.fit_substrate_logistic
            self.fit_product = self.fit_product_logistic
        elif model_type == 'gompertz':
            self.fit_biomass = self.fit_biomass_gompertz
            self.fit_substrate = self.fit_substrate_gompertz
            self.fit_product = self.fit_product_gompertz
        elif model_type == 'moser':
            self.fit_biomass = self.fit_biomass_moser
            self.fit_substrate = self.fit_substrate_moser
            self.fit_product = self.fit_product_moser
        else:
            raise ValueError("Invalid model type. Choose 'logistic', 'gompertz', or 'moser'.")

    def fit_biomass_logistic(self, time, biomass):
        popt, _ = curve_fit(self.logistic, time, biomass, bounds=(0, [np.inf, np.inf, np.inf]), maxfev=10000)
        self.params['biomass'] = {'xo': popt[0], 'xm': popt[1], 'um': popt[2]}
        y_pred = self.logistic(time, *popt)
        self.r2['biomass'] = 1 - (np.sum((biomass - y_pred)**2) / np.sum((biomass - np.mean(biomass))**2))
        self.rmse['biomass'] = np.sqrt(mean_squared_error(biomass, y_pred))
        return y_pred

    def fit_substrate_logistic(self, time, substrate, biomass_params):
        popt, _ = curve_fit(lambda t, so, p, q: self.substrate(t, so, p, q, *biomass_params.values()),
                            time, substrate, bounds=(0, [np.inf, np.inf, np.inf]))
        self.params['substrate'] = {'so': popt[0], 'p': popt[1], 'q': popt[2]}
        y_pred = self.substrate(time, *popt, *biomass_params.values())
        self.r2['substrate'] = 1 - (np.sum((substrate - y_pred)**2) / np.sum((substrate - np.mean(substrate))**2))
        self.rmse['substrate'] = np.sqrt(mean_squared_error(substrate, y_pred))
        return y_pred

    def fit_product_logistic(self, time, product, biomass_params):
        popt, _ = curve_fit(lambda t, po, alpha, beta: self.product(t, po, alpha, beta, *biomass_params.values()),
                            time, product, bounds=(0, [np.inf, np.inf, np.inf]))
        self.params['product'] = {'po': popt[0], 'alpha': popt[1], 'beta': popt[2]}
        y_pred = self.product(time, *popt, *biomass_params.values())
        self.r2['product'] = 1 - (np.sum((product - y_pred)**2) / np.sum((product - np.mean(product))**2))
        self.rmse['product'] = np.sqrt(mean_squared_error(product, y_pred))
        return y_pred

    def fit_biomass_gompertz(self, time, biomass):
        popt, _ = curve_fit(self.gompertz_biomass, time, biomass, bounds=(0, [np.inf, np.inf, np.inf, 10]), maxfev=10000)
        self.params['biomass'] = {'xo': popt[0], 'lbd': popt[1], 'um': popt[2], 'bg': popt[3]}
        y_pred = self.gompertz_biomass(time, *popt)
        self.r2['biomass'] = 1 - (np.sum((biomass - y_pred)**2) / np.sum((biomass - np.mean(biomass))**2))
        self.rmse['biomass'] = np.sqrt(mean_squared_error(biomass, y_pred))
        return y_pred

    def fit_substrate_gompertz(self, time, substrate, biomass_params):
        popt, _ = curve_fit(lambda t, so, po, yps: self.gompertz_substrate(t, so, po, yps, biomass_params['bg'], biomass_params['um']),
                            time, substrate, bounds=(0, [np.inf, np.inf, np.inf]))
        self.params['substrate'] = {'so': popt[0], 'po': popt[1], 'yps': popt[2]}
        y_pred = self.gompertz_substrate(time, *popt, biomass_params['bg'], biomass_params['um'])
        self.r2['substrate'] = 1 - (np.sum((substrate - y_pred)**2) / np.sum((substrate - np.mean(substrate))**2))
        self.rmse['substrate'] = np.sqrt(mean_squared_error(substrate, y_pred))
        return y_pred

    def fit_product_gompertz(self, time, product, biomass_params):
        popt, _ = curve_fit(lambda t, po, cg: self.gompertz_product(t, po, biomass_params['um'], cg),
                            time, product, bounds=(0, [np.inf, np.inf]))
        self.params['product'] = {'po': popt[0], 'cg': popt[1]}
        y_pred = self.gompertz_product(time, *popt, biomass_params['um'])
        self.r2['product'] = 1 - (np.sum((product - y_pred)**2) / np.sum((product - np.mean(product))**2))
        self.rmse['product'] = np.sqrt(mean_squared_error(product, y_pred))
        return y_pred

    def fit_biomass_moser(self, time, biomass, sustrato):
        popt, _ = curve_fit(lambda t, xo, umax, ks, n: self.moser_biomass(t, xo, umax, ks, n, sustrato),
                            time, biomass, bounds=(0, [np.inf, 1, np.inf, np.inf]), maxfev=10000)
        self.params['biomass'] = {'xo': popt[0], 'umax': popt[1], 'ks': popt[2], 'n': popt[3]}
        y_pred = self.moser_biomass(time, *popt, sustrato)
        self.r2['biomass'] = 1 - (np.sum((biomass - y_pred)**2) / np.sum((biomass - np.mean(biomass))**2))
        self.rmse['biomass'] = np.sqrt(mean_squared_error(biomass, y_pred))
        return y_pred

    def fit_substrate_moser(self, time, substrate, biomass_params, sustrato):
        popt, _ = curve_fit(lambda t, so, p: self.moser_substrate(t, so, p, biomass_params['xo'], biomass_params['umax'], biomass_params['ks'], biomass_params['n'], sustrato),
                            time, substrate, bounds=(0, [np.inf, np.inf]))
        self.params['substrate'] = {'so': popt[0], 'p': popt[1]}
        y_pred = self.moser_substrate(time, *popt, biomass_params['xo'], biomass_params['umax'], biomass_params['ks'], biomass_params['n'], sustrato)
        self.r2['substrate'] = 1 - (np.sum((substrate - y_pred)**2) / np.sum((substrate - np.mean(substrate))**2))
        self.rmse['substrate'] = np.sqrt(mean_squared_error(substrate, y_pred))
        return y_pred

    def fit_product_moser(self, time, product, biomass_params, sustrato):
        popt, _ = curve_fit(lambda t, po, p: self.moser_product(t, po, p, biomass_params['xo'], biomass_params['umax'], biomass_params['ks'], biomass_params['n'], sustrato),
                            time, product, bounds=(0, [np.inf, np.inf]))
        self.params['product'] = {'po': popt[0], 'p': popt[1]}
        y_pred = self.moser_product(time, *popt, biomass_params['xo'], biomass_params['umax'], biomass_params['ks'], biomass_params['n'], sustrato)
        self.r2['product'] = 1 - (np.sum((product - y_pred)**2) / np.sum((product - np.mean(product))**2))
        self.rmse['product'] = np.sqrt(mean_squared_error(product, y_pred))
        return y_pred

In [ ]:
# Celda 4: Método de visualización de resultados
class BioprocessModel(BioprocessModel):  # Continuación de la clase
    def plot_results(self, time, biomass, substrate, product, y_pred_biomass, y_pred_substrate, y_pred_product, experiment_name, legend_position, params_position):
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15))
        fig.suptitle(f'Tratamiento: {experiment_name}', fontsize=16)

        plots = [
            (ax1, biomass, y_pred_biomass, 'Biomasa', 'Modelo seleccionado', self.params['biomass'], self.r2['biomass'], self.rmse['biomass']),
            (ax2, substrate, y_pred_substrate, 'Sustrato', 'Modelo seleccionado', self.params['substrate'], self.r2['substrate'], self.rmse['substrate']),
            (ax3, product, y_pred_product, 'Producto', 'Modelo seleccionado', self.params['product'], self.r2['product'], self.rmse['product'])
        ]

        for ax, data, y_pred, ylabel, model_name, params, r2, rmse in plots:
            ax.plot(time, data, 'o', label='Datos experimentales')
            ax.plot(time, y_pred, '-', label=model_name)
            ax.set_xlabel('Tiempo')
            ax.set_ylabel(ylabel)
            ax.legend(loc=legend_position)
            ax.set_title(f'{model_name} - {ylabel}')

            param_text = '\n'.join([f"{k} = {v:.4f}" for k, v in params.items()])
            text = f"{param_text}\nR² = {r2:.4f}\nRMSE = {rmse:.4f}"

            if params_position in ['upper right', 'lower right']:
                text_x = 0.95
                ha = 'right'
            else:
                text_x = 0.05
                ha = 'left'

            if params_position in ['upper right', 'upper left']:
                text_y = 0.95
                va = 'top'
            else:
                text_y = 0.05
                va = 'bottom'

            ax.text(text_x, text_y, text, transform=ax.transAxes,
                    verticalalignment=va, horizontalalignment=ha,
                    bbox={'boxstyle':'round', 'facecolor':'white', 'alpha':0.5})

        plt.tight_layout()
        return fig

In [ ]:
# Celda 5: Función de procesamiento de datos
def process_data(file, legend_position, params_position, model_type, experiment_names):
    # Leer todas las hojas del archivo Excel
    xls = pd.ExcelFile(file.name)
    sheet_names = xls.sheet_names

    model = BioprocessModel()
    model.fit_model(model_type)
    figures = []

    for i, sheet_name in enumerate(sheet_names, 1):
        df = pd.read_excel(file.name, sheet_name=sheet_name, header=[0, 1])

        # Procesar datos
        model.process_data(df)

        # Extraer datos promedio para el ajuste del modelo
        time = df[('A', 'Tiempo')].values
        biomass = model.dataxp[i-1]
        substrate = model.datasp[i-1]
        product = model.datapp[i-1]

        if model_type == 'moser':
            y_pred_biomass = model.fit_biomass(time, biomass, substrate)
            y_pred_substrate = model.fit_substrate(time, substrate, model.params['biomass'], substrate)
            y_pred_product = model.fit_product(time, product, model.params['biomass'], substrate)
        else:
            y_pred_biomass = model.fit_biomass(time, biomass)
            y_pred_substrate = model.fit_substrate(time, substrate, model.params['biomass'])
            y_pred_product = model.fit_product(time, product, model.params['biomass'])

        # Usar el nombre del experimento proporcionado o un nombre por defecto
        experiment_name = experiment_names[i-1] if i-1 < len(experiment_names) else f"Tratamiento {i}"

        fig = model.plot_results(time, biomass, substrate, product, y_pred_biomass, y_pred_substrate, y_pred_product, experiment_name, legend_position, params_position)
        figures.append(fig)

    return figures

In [ ]:
# Celda 6: Creación de la interfaz de Gradio
def create_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# Modelos de Bioproceso: Logístico, Gompertz y Moser")
        gr.Markdown("Sube un archivo Excel con múltiples pestañas. Cada pestaña debe contener columnas 'Tiempo', 'Biomasa', 'Sustrato' y 'Producto' para cada experimento (A, B, C).")

        file_input = gr.File(label="Subir archivo Excel")

        with gr.Row():
            with gr.Column():
                gr.Markdown("## Posición de la Leyenda")
                legend_ul = gr.Checkbox(label="Arriba Izquierda", value=True)
                legend_ur = gr.Checkbox(label="Arriba Derecha")
                legend_ll = gr.Checkbox(label="Abajo Izquierda")
                legend_lr = gr.Checkbox(label="Abajo Derecha")

            with gr.Column():
                gr.Markdown("## Posición de los Parámetros")
                params_ul = gr.Checkbox(label="Arriba Izquierda", value=True)
                params_ur = gr.Checkbox(label="Arriba Derecha")
                params_ll = gr.Checkbox(label="Abajo Izquierda")
                params_lr = gr.Checkbox(label="Abajo Derecha")

        model_type = gr.Radio(["logistic", "gompertz", "moser"], label="Tipo de Modelo", value="logistic")

        experiment_names = gr.Textbox(
            label="Nombres de los experimentos (uno por línea)",
            placeholder="Experimento 1\nExperimento 2\n...",
            lines=5
        )

        simulate_btn = gr.Button("Simular")

        output_plots = [gr.Plot(label=f"Gráfico {i+1}") for i in range(10)] ########################################CAMBIAR DE ACUERDO A NUMERO DE HOJAS DE EXCEL PARA EXPERIMENTOS

        def process_and_plot(file, legend_ul, legend_ur, legend_ll, legend_lr,
                             params_ul, params_ur, params_ll, params_lr, model_type, experiment_names):
            legend_pos = 'upper left'
            if legend_ur:
                legend_pos = 'upper right'
            elif legend_ll:
                legend_pos = 'lower left'
            elif legend_lr:
                legend_pos = 'lower right'

            params_pos = 'upper left'
            if params_ur:
                params_pos = 'upper right'
            elif params_ll:
                params_pos = 'lower left'
            elif params_lr:
                params_pos = 'lower right'

            # Dividir los nombres de experimentos en una lista
            experiment_names_list = experiment_names.split('\n') if experiment_names else []

            return process_data(file, legend_pos, params_pos, model_type, experiment_names_list)

        simulate_btn.click(
            fn=process_and_plot,
            inputs=[file_input,
                    legend_ul, legend_ur, legend_ll, legend_lr,
                    params_ul, params_ur, params_ll, params_lr,
                    model_type,
                    experiment_names],
            outputs=output_plots
        )

    return demo

In [ ]:
# Celda 7: Funciones auxiliares y lanzamiento de la interfaz
global_interface = None

def get_interface():
    global global_interface
    if global_interface is None:
        global_interface = create_interface()
    return global_interface

# Crear y lanzar la interfaz
demo = get_interface()
demo.launch()